# AudioToText Pipeline Demo

## This notebook demonstrates how to use the TextToAudio pipeline to convert text into speech using a text-to-speech model.


In [1]:
import logging
from datasets import load_dataset
from huggingface_pipelines.pipeline_config import AudioPipelineConfig, DatasetConfig
from huggingface_pipelines.speech import AudioToTextHFPipeline
import matplotlib.pyplot as plt
import librosa
import librosa.display
import IPython.display as ipd
import numpy as np

logging.basicConfig(level=logging.INFO)



/Users/botirkhaltaev/Desktop/SONAR/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_config = DatasetConfig(
        dataset_name="common_voice",
        dataset_split="test",
        trust_remote_code = True,
        config = "ab",
    )

dataset = dataset_config.load_dataset()

TypeError: BuilderConfig.__init__() got an unexpected keyword argument 'config'

## Configure Pipeline

In [ ]:
config = AudioPipelineConfig(
    dataset_name="common_voice",
    columns=["audio"]
).with_overwrites({
    "encoder_model": "sonar_speech_encoder_eng",
    "decoder_model": "text_sonar_basic_decoder",
    "target_lang": "eng_Latn",
    "batch_size": 16,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "output_file_name": "transcription_results",
    "take": 100
})

## Initialize and Run Pipeline

In [ ]:
pipeline = AudioToTextHFPipeline(config)
dataset = pipeline(dataset)

## Display Results

In [ ]:
for original, transcribed in zip(dataset['sentence'][:5], processed_dataset['audio_transcribed'][:5]):
    print(f"Original: {original}")
    print(f"Transcribed: {transcribed}")
    print()


## Visualize and Play Audio Sample

In [ ]:
def plot_audio_and_transcription(audio, transcription, sample_rate):
    plt.figure(figsize=(12, 4))
    librosa.display.waveshow(audio, sr=sample_rate)
    plt.title(f'Audio Waveform\nTranscription: {transcription}')
    plt.xlabel('Time')
    plt.ylabel('Amplitude')
    plt.show()
    
    ipd.display(ipd.Audio(audio, rate=sample_rate))

sample_audio = dataset['audio'][0]['array']
sample_rate = dataset['audio'][0]['sampling_rate']
sample_transcription = processed_dataset['audio_transcribed'][0]

plot_audio_and_transcription(sample_audio, sample_transcription, sample_rate)

## Configure and Run MetricAnalyzer Pipeline

In [ ]:
metric_config = MetricPipelineConfig(
    dataset_name="common_voice",
    dataset_split="test",
    columns=["audio_transcribed"]
).with_overwrites({
    "batch_size": 32,
    "device": "cpu",
    "metric_name": "wer",  # Word Error Rate
    "low_score_threshold": 0.5,  # Adjust as needed
    "output_file_name": "transcription_metrics",
    "take": 100
})

In [ ]:
metric_pipeline = MetricAnalyzerPipeline(metric_config)
dataset = metric_pipeline(transcribed_dataset)

## Analyze Metric Scores

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(evaluated_dataset['wer'], bins=20)
plt.title('Distribution of Word Error Rates')
plt.xlabel('Word Error Rate')
plt.ylabel('Frequency')
plt.axvline(x=metric_config.low_score_threshold, color='r', linestyle='--', label='Low Score Threshold')
plt.legend()
plt.show()

print(f"Average WER: {sum(evaluated_dataset['wer']) / len(evaluated_dataset['wer']):.4f}")


## Analyze Low-Scoring Samples

In [ ]:
low_scoring = evaluated_dataset.filter(lambda x: x['wer'] > metric_config.low_score_threshold)
print(f"Number of low-scoring samples: {len(low_scoring)}")

for sample in low_scoring[:5]:
    print(f"Original: {sample['sentence']}")
    print(f"Transcribed: {sample['audio_transcribed']}")
    print(f"WER: {sample['wer']:.4f}")
    print()